# ML4DD Summer School Hackathon

The final days of the Machine Learning For Drug Discovery summer school ends with a hackathon. We will use Polaris as a tool to get the associated benchmarks and datasets. First things first, we will install Polaris from PyPi.

We next need to authenticate ourselves to Polaris. If you haven't done so yet, you can create an account at https://polarishub.io. Afterwards, you can simply run the command below.

In [1]:
# Use the organization owner settings
owner = "team13"

print(f'You have set "{owner}" as the owner')

You have set "team13" as the owner


In [2]:
import polaris as po
import datamol as dm
import numpy as np
import pandas as pd
import torch

torch.manual_seed(42)
np.random.seed(42)

/Users/dre/Desktop/GitRepo/DeAddicts/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Kinase Selectivity

The second benchmark we will use is `polaris/pkis1-kit-wt-mut-c-1`. Using this benchmark is very similar to before, except for one difference: This is a multi-task benchmark.

In [3]:
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split()

2024-06-20 20:19:22.857 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-20 20:19:22.868 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


As we can see, the targets are now returned to us as a dictionary. Let's train a multi-task model on this data! We first preprocess the data to be in a format we can use with scikit-learn.

In [4]:
ys = train.y
ys = np.stack([ys[target] for target in benchmark.target_cols], axis=1)
ys.shape

(277, 3)

Now that we're working with a multi-task dataset, it's also possible for these arrays to be sparse. Let's filter out any data points that doesn't have readouts for _all_ targets.

In [5]:
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()

276

In [6]:
df_train = pd.DataFrame(train.X[mask])
df_train.columns = ["smiles"]
df_train[benchmark.target_cols] = ys[mask]

In [7]:
df_train.head()

,smiles,CLASS_KIT_(T6701_mutant),CLASS_KIT_(V560G_mutant),CLASS_KIT
0,O=C(Nc1n[nH]c2cc(-c3ccc(F)cc3)ccc12)C1CC1,0.0,0.0,0.0
1,CCn1c(-c2nonc2N)nc2c(C#CC(C)(C)O)ncc(OC3CCNCC3...,0.0,0.0,0.0
2,CN(C)c1cc2c(Nc3ccc4c(cnn4Cc4ccccc4)c3)ncnc2cn1,0.0,0.0,0.0
3,NS(=O)(=O)c1cccc(-c2ccc3c(NC(=O)C4CC4)n[nH]c3c...,0.0,0.0,0.0
4,Cc1nn(C)c2cc(N(C)c3ccnc(Nc4cccc(S(N)(=O)=O)c4)...,0.0,0.0,1.0


## Add physical features

In [8]:
features = [
    "MolecularWeight",
    "LogP",
    "MaxAbsPartialCharge",
    "MinAbsPartialCharge",
]

In [9]:
from src.utils import featurize_smiles

df_train[features] = df_train["smiles"].apply(lambda x: pd.Series(featurize_smiles(x)))


# Add fingerprints

In [10]:

fp = []
for smile in df_train["smiles"]:
    fp.append(dm.to_fp(smile))
    
fp = pd.DataFrame(fp, index=df_train.index)
df_train = pd.concat([df_train, fp], axis=1)

In [11]:
df_train.head()

,smiles,CLASS_KIT_(T6701_mutant),CLASS_KIT_(V560G_mutant),CLASS_KIT,MolecularWeight,LogP,MaxAbsPartialCharge,MinAbsPartialCharge,0,1,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,O=C(Nc1n[nH]c2cc(-c3ccc(F)cc3)ccc12)C1CC1,0.0,0.0,0.0,295.317,3.71750,0.308459,0.228201,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CCn1c(-c2nonc2N)nc2c(C#CC(C)(C)O)ncc(OC3CCNCC3...,0.0,0.0,0.0,411.466,1.33660,0.486426,0.199153,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CN(C)c1cc2c(Nc3ccc4c(cnn4Cc4ccccc4)c3)ncnc2cn1,0.0,0.0,0.0,395.470,4.23240,0.362725,0.141380,0,0,...,0,0,0,0,0,0,0,0,1,0
3,NS(=O)(=O)c1cccc(-c2ccc3c(NC(=O)C4CC4)n[nH]c3c...,0.0,0.0,0.0,356.407,2.22580,0.308459,0.237567,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cc1nn(C)c2cc(N(C)c3ccnc(Nc4cccc(S(N)(=O)=O)c4)...,0.0,0.0,1.0,423.502,2.83062,0.329238,0.237623,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X = df_train.drop(columns=["smiles"] + benchmark.target_cols).values
y = df_train[benchmark.target_cols].values

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (276, 2052)
y shape: (276, 3)


Do the same featurization on the test set.

In [13]:
df_test = pd.DataFrame(test.X)
df_test.columns = ["smiles"]

df_test[features] = df_test["smiles"].apply(lambda x: pd.Series(featurize_smiles(x)))

fp = []
for smile in df_test["smiles"]:
    fp.append(dm.to_fp(smile))

fp = pd.DataFrame(fp, index=df_test.index)
df_test = pd.concat([df_test, fp], axis=1)

X_test = df_test.drop(columns=["smiles"]).values

# Baseline with Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
# Construct a random forest regressor for each target
models = {
    target: RandomForestClassifier(max_depth=5) for target in benchmark.target_cols
}

# Train the models
for target in benchmark.target_cols:
    models[target].fit(X, y[:, benchmark.target_cols.index(target)])

## Predictions

In [16]:
# Predict the test set
y_prob_rf = {
    target: model.predict_proba(X_test)[:, 1] for target, model in models.items()
}
y_pred_rf = {target: model.predict(X_test) for target, model in models.items()}

In [17]:
results = benchmark.evaluate(y_pred=y_pred_rf, y_prob=y_prob_rf)
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6206896552
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.668297456
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7172222222
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.8159371493
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6049975606
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.5444855267
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6839259998
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.0
    }
  ]
}

# Multioutput

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [19]:
base_model = RandomForestClassifier()
model = MultiOutputClassifier(base_model)

In [20]:
model.fit(X, y)

MultiOutputClassifier(estimator=RandomForestClassifier())

## Predictions

In [21]:
# Predict the test set
y_pred_mrf = model.predict(X_test)
y_prob_mrf = model.predict_proba(X_test)

In [22]:
y_pred_mrf = {
    target: y_pred_mrf[:, i] for i, target in enumerate(benchmark.target_cols)
}
y_prob_mrf = {
    target: y_prob_mrf[i][:, 1] for i, target in enumerate(benchmark.target_cols)
}

In [23]:
print(benchmark.target_cols)

['CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)', 'CLASS_KIT']


In [24]:
# Predict for class 1
print("Predicted as binders for target 1:", y_pred_mrf[benchmark.target_cols[0]].sum())
print("Predicted as binders for target 2:", y_pred_mrf[benchmark.target_cols[1]].sum())
print("Predicted as binders for target 3:", y_pred_mrf[benchmark.target_cols[2]].sum())

Predicted as binders for target 1: 0.0
Predicted as binders for target 2: 0.0
Predicted as binders for target 3: 2.0


In [25]:
results = benchmark.evaluate(y_pred=y_pred_mrf, y_prob=y_prob_mrf)
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6436781609
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.1142857143
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.6863992172
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.6933333333
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.8010662177
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.3667219285
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4745649745
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6948023478
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.1962209821
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.0741503605
    }
  ]
}

# Resampling

In [26]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

resampled_datasets = {}

for target in benchmark.target_cols:
    X_resampled, y_resampled = smote.fit_resample(
        X, y[:, benchmark.target_cols.index(target)]
    )
    resampled_datasets[target] = (X_resampled, y_resampled)

In [27]:
models = {
    target: RandomForestClassifier(max_depth=5, n_estimators=100) for target in benchmark.target_cols
}

In [28]:
for target in benchmark.target_cols:
    models[target].fit(*resampled_datasets[target])

## Predictions

In [29]:
y_pred_rf_resampled = {
    target: model.predict(X_test) for target, model in models.items()
}
y_prob_rf_resampled = {
    target: model.predict_proba(X_test)[:, 1] for target, model in models.items()
}

In [30]:
# Predict for class 1
print(
    "Predicted as binders for target 1:",
    y_pred_rf_resampled[benchmark.target_cols[0]].sum(),
)
print(
    "Predicted as binders for target 2:",
    y_pred_rf_resampled[benchmark.target_cols[1]].sum(),
)
print(
    "Predicted as binders for target 3:",
    y_pred_rf_resampled[benchmark.target_cols[2]].sum(),
)

Predicted as binders for target 1: 6.0
Predicted as binders for target 2: 3.0
Predicted as binders for target 3: 6.0


In [31]:
results = benchmark.evaluate(y_pred=y_pred_rf_resampled, y_prob=y_prob_rf_resampled)
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.908045977
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8735632184
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6436781609
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.6
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.2666666667
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.2051282051
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7358121331
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.715
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.7424242424
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6685591099
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4241572522
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6287281486
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.6214848238
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.2897727626
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.1611830148
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.5572519084
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.2238442822
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.1001001001
    }
  ]
}

# XGBoost with resampling

In [ ]:
from xgboost import XGBClassifier

In [ ]:
models = {
    target: XGBClassifier(max_depth=5, n_estimators=100) for target in benchmark.target_cols
}

In [ ]:
for target in benchmark.target_cols:
    models[target].fit(*resampled_datasets[target])

## Predictions

In [ ]:
y_pred_xgb_resampled = {
    target: model.predict(X_test) for target, model in models.items()
}
y_prob_xgb_resampled = {
    target: model.predict_proba(X_test)[:, 1] for target, model in models.items()
}

In [ ]:
# Predict for class 1
print(
    "Predicted as binders for target 1:",
    y_pred_xgb_resampled[benchmark.target_cols[0]].sum(),
)
print(
    "Predicted as binders for target 2:",
    y_pred_xgb_resampled[benchmark.target_cols[1]].sum(),
)
print(
    "Predicted as binders for target 3:",
    y_pred_xgb_resampled[benchmark.target_cols[2]].sum(),
)

In [ ]:
results = benchmark.evaluate(y_pred=y_pred_xgb_resampled, y_prob=y_prob_xgb_resampled)
results

# MLP with weighted loss

In [32]:
# Define an MLP model, then train it with a loss which is proportional to the class imbalance

import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, hidden_layers[0]))
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))
        self.layers.append(nn.Linear(hidden_layers[-1], output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        return x


class WeightedBCEWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight):
        super(WeightedBCEWithLogitsLoss, self).__init__()
        self.pos_weight = pos_weight

    def forward(self, y_pred, y_true):
        criterion = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        return criterion(y_pred, y_true)

In [33]:
weights = {}

for target in benchmark.target_cols:
    num_positive = y[:, benchmark.target_cols.index(target)].sum()
    num_negative = y.shape[0] - num_positive
    pos_weight = num_negative / num_positive
    weights[target] = torch.tensor(pos_weight)

In [34]:
n_epochs = 30
hidden_layers = [[32, 16] for _ in benchmark.target_cols]

# Define the model
models = {
    target: MLP(input_dim=X.shape[1], hidden_layers=hidden_layers[i], output_dim=1)
    for i, target in enumerate(benchmark.target_cols)
}

for target in benchmark.target_cols:
    model = models[target]
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = WeightedBCEWithLogitsLoss(pos_weight=weights[target])

    X_t = torch.tensor(X, dtype=torch.float32)
    y_t = torch.tensor(y[:, benchmark.target_cols.index(target)], dtype=torch.float32)

    for epoch in range(n_epochs):
        optimizer.zero_grad()
        y_pred = model(X_t).squeeze()
        loss = criterion(y_pred, y_t)
        loss.backward()
        optimizer.step()

    model.eval()

## Predict

In [35]:
# Predict the test set
y_prob_mlp = {
    target: model(torch.tensor(X_test, dtype=torch.float32)).detach().numpy().squeeze()
    for target, model in models.items()
}

y_pred_mlp = {
    target: (y_prob > 0.5).astype(int) for target, y_prob in y_prob_mlp.items()
}

In [36]:
print("Predicted as binders for target 1:", y_pred_mlp[benchmark.target_cols[0]].sum())
print("Predicted as binders for target 2:", y_pred_mlp[benchmark.target_cols[1]].sum())
print("Predicted as binders for target 3:", y_pred_mlp[benchmark.target_cols[2]].sum())

Predicted as binders for target 1: 7
Predicted as binders for target 2: 8
Predicted as binders for target 3: 18


In [37]:
results = benchmark.evaluate(y_pred=y_pred_mlp, y_prob=y_prob_mlp)
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.9195402299
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8850574713
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.7356321839
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.6666666667
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.5
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.5490196078
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7915851272
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.8211111111
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.8310886644
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6788662544
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.5259101953
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.7451777898
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.6754628043
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.4494898656
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.4194400939
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.6266094421
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.4379844961
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.3841181902
    }
  ]
}

## Upload results

In [ ]:
results.name = "my-second-result"
results.description = "ECFP fingerprints with a Random Forest"

In [ ]:
# results.upload_to_hub(owner=owner)